In [1]:
import os
import sys
import re
import pandas as pd

In [2]:
pattern = r"(?<=output_).[^_]*(?=_)"
dir_name = "outputs_science"
output_file_name = "science_videos_Aug132023.csv"
channel_list = set()
files = [f"{dir_name}/{f}" for f in os.listdir(f"{dir_name}/") if f.endswith(".csv")]

for f in files:
    channel_name = re.search(pattern, f)[0]
    channel_list.add(channel_name)
    print(f)


outputs_science/output_2veritasium_0.csv
outputs_science/output_2veritasium_1.csv
outputs_science/output_365DataScience_0.csv
outputs_science/output_365DataScience_1.csv
outputs_science/output_3Blue1Brown_0.csv
outputs_science/output_3Blue1Brown_1.csv
outputs_science/output_3x3Custom_0.csv
outputs_science/output_3x3Custom_1.csv
outputs_science/output_AboveTheNoise_0.csv
outputs_science/output_AboveTheNoise_1.csv
outputs_science/output_AlexanderAmini_0.csv
outputs_science/output_AlexanderAmini_1.csv
outputs_science/output_AppliedScience_0.csv
outputs_science/output_AppliedScience_1.csv
outputs_science/output_AsapSCIENCE_0.csv
outputs_science/output_AsapSCIENCE_1.csv
outputs_science/output_Astrolabe_0.csv
outputs_science/output_Astrolabe_1.csv
outputs_science/output_Astrum_0.csv
outputs_science/output_Astrum_1.csv
outputs_science/output_BenEater_0.csv
outputs_science/output_BenEater_1.csv
outputs_science/output_BeSmart_0.csv
outputs_science/output_BeSmart_1.csv
outputs_science/output_Bit

In [3]:
print(len(channel_list))
channel_list = list(channel_list)
channel_list[0]

138


'LauraKampf'

In [4]:
all_df_list = []
for channel in channel_list:
    channel_files = [f for f in files if f.startswith(f"{dir_name}/output_{channel}")]
    channel_df_list = []

    for f in channel_files:
        df_temp = pd.read_csv(f, encoding= 'unicode_escape')
        channel_df_list.append(df_temp)

    df = pd.concat(channel_df_list).drop_duplicates(subset=["video_id"], keep='first').reset_index(drop=True)
    all_df_list.append(df)
    
df = pd.DataFrame()
df = pd.concat(all_df_list).drop_duplicates(subset=["video_id"], keep='first').reset_index(drop=True)
df

,video_id,channel_name,video_title
0,aZ4MKhqvz2w,LauraKampf,LauraÂ´s Beer Bike (BMX meets Handtruck)
1,vOEUttzRjkE,LauraKampf,Building Stairs in 8 Hours
2,bEnjDkiQ22A,LauraKampf,Camper Renovation on a Budget! (no money spent...
3,InDQBhEJgkQ,LauraKampf,LauraÂ´s Steel Bender
4,3pLmbbQWVe4,LauraKampf,Building a solid wood door (with clever features)
...,...,...,...
27692,nAWVYFEzoY8,CoreySchafer,"JavaScript Arrays: Properties, Methods, and Ma..."
27693,jF-OQ-BrIAM,CoreySchafer,CSS Tips and Tricks: Add External URLs to Prin...
27694,w4KF_lapbRI,CoreySchafer,"JavaScript Arrays: Properties, Methods, and Ma..."
27695,4qMJN1pY_aw,CoreySchafer,"Channel Update: Code Snippets, New Rewards, an..."


In [5]:
df.to_csv(output_file_name, index=False, encoding="utf-8")